<a href="https://colab.research.google.com/github/DanielaMorariu1990/Generate_image_captions/blob/main/eJobs_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import os
import pandas as pd
import numpy as np
import keras
import re
import string
from nltk.corpus import stopwords


In [67]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [102]:
! pip install transformers

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 6.6 MB 34.5 MB/s 
     |████████████████████████████████| 596 kB 17.1 MB/s 
     |████████████████████████████████| 895 kB 15.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [106]:

from sklearn.model_selection import train_test_split

from keras.layers import Dense,Dropout, Input

import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig

In [1]:
!pip install kaggle

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [3]:
!kaggle datasets download "vbmokin/nlp-reports-news-classification/"

  0% 0.00/16.0k [00:00<?, ?B/s]
100% 16.0k/16.0k [00:00<00:00, 14.8MB/s]


In [4]:
!unzip nlp-reports-news-classification.zip

Archive:  nlp-reports-news-classification.zip
  inflating: BUWR-SB-basin-water-resources.csv  
  inflating: water_problem_nlp_en_for_Kaggle_100.csv  
  inflating: water_problem_nlp_ua_for_Kaggle_100.csv  


In [6]:
import os
print(os.listdir("."))

['.config', 'BUWR-SB-basin-water-resources.csv', 'water_problem_nlp_en_for_Kaggle_100.csv', 'water_problem_nlp_ua_for_Kaggle_100.csv', 'nlp-reports-news-classification.zip', 'sample_data']


In [20]:
basin_water=pd.read_csv("BUWR-SB-basin-water-resources.csv")
water_problems=pd.read_csv("water_problem_nlp_en_for_Kaggle_100.csv", sep=";")
w_prob_ua=pd.read_csv("water_problem_nlp_ua_for_Kaggle_100.csv",encoding='cp1251', sep=";")

In [24]:
water_problems.head()

,text,env_problems,pollution,treatment,climate,biomonitoring
0,In the Southern Bug basin wastewater is discha...,1,1.0,NaN,NaN,NaN
1,This amount is distributed as follows: urban a...,1,1.0,NaN,NaN,NaN
2,Despite the similar volumes of discharged wast...,1,1.0,NaN,NaN,NaN
3,They bring 84% of organic pollution 86% of pho...,1,1.0,NaN,NaN,NaN
4,The input of the industry is between 7–21% and...,1,1.0,NaN,NaN,NaN


In [25]:
water_problems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   text           100 non-null    object 
 1   env_problems   100 non-null    int64  
 2   pollution      38 non-null     float64
 3   treatment      18 non-null     float64
 4   climate        19 non-null     float64
 5   biomonitoring  8 non-null      float64
dtypes: float64(4), int64(1), object(1)
memory usage: 4.8+ KB


In [26]:
water_problems.describe()

,env_problems,pollution,treatment,climate,biomonitoring
count,100.000000,38.0,18.0,19.0,8.0
mean,0.510000,1.0,1.0,1.0,1.0
std,0.502418,0.0,0.0,0.0,0.0
min,0.000000,1.0,1.0,1.0,1.0
25%,0.000000,1.0,1.0,1.0,1.0
50%,1.000000,1.0,1.0,1.0,1.0
75%,1.000000,1.0,1.0,1.0,1.0
max,1.000000,1.0,1.0,1.0,1.0


In [29]:
water_problems.replace(np.nan,0, inplace=True)

In [30]:
water_problems.head()

,text,env_problems,pollution,treatment,climate,biomonitoring
0,In the Southern Bug basin wastewater is discha...,1,1.0,0.0,0.0,0.0
1,This amount is distributed as follows: urban a...,1,1.0,0.0,0.0,0.0
2,Despite the similar volumes of discharged wast...,1,1.0,0.0,0.0,0.0
3,They bring 84% of organic pollution 86% of pho...,1,1.0,0.0,0.0,0.0
4,The input of the industry is between 7–21% and...,1,1.0,0.0,0.0,0.0


In [34]:
water_problems["text"][0]

'In the Southern Bug basin wastewater is discharged from 341 organized sources (Map 5). In 2011 (reference year) 211 mln. m3 was discharged in total.'

In [37]:
water_problems.loc[10]

text             The degree of connectivity of the population t...
env_problems                                                     1
pollution                                                      0.0
treatment                                                      1.0
climate                                                        0.0
biomonitoring                                                  0.0
Name: 10, dtype: object

# Clean data

* eliminate punctuations,white space, numbers, very common words such as "the", "and" , 'in", "a" etc --> they do not add menaing to the text 
* for the stop words I use NLTK corpus which contains 127 stopwords in the english language
* we do not need to trasnform everything to lower case becuase case is not taken into account by the bert algorithm that we will use to get the word embeddings (however we do, because stop words are case sensitive :) )


In [82]:
stop_words=set(stopwords.words('english'))
def clean_text(text):
    text = [i.translate(str.maketrans('', '', string.punctuation))
               for i in text.split()]
    text = [i.lower() for i in text]
    text = [i.rstrip() for i in text]
    text = [i.lstrip() for i in text]
    text= list(filter(lambda w: not w in stop_words,text))
    text = ' '.join([i for i in text if not i.isdigit()])
    return text

In [85]:
water_problems["clean_text"]=water_problems.text.apply(clean_text)

In [87]:
water_problems["clean_text"][0]

'southern bug basin wastewater discharged organized sources map reference year mln m3 discharged total'

In [88]:
water_problems["text"][0]

'In the Southern Bug basin wastewater is discharged from 341 organized sources (Map 5). In 2011 (reference year) 211 mln. m3 was discharged in total.'

# Get word (sentance) embeddings from Bert algorithm

In [95]:
## get max lengths of the sentances
def get_length_of_sentance(sentance):
  return len(sentance.split())

In [99]:
max_length=water_problems.clean_text.apply(get_length_of_sentance).max()
max_length

31

In [107]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpxoz489nr


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp0ujpnoui


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbe

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
creating metadata file for /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.0

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 in cache at /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
creating metadata file for /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the

In [217]:
def preprocessing_data(df_column, max_length=31):
  all_input=[]
  attention_mask=[]
  for sentance in df_column:
    inputs=tokenizer.encode_plus(sentance,add_special_tokens = True,padding='max_length',truncation=True)
    all_input.append(inputs["input_ids"])
    attention_mask.append(inputs["attention_mask"])
  return np.array(all_input), np.array(attention_mask)



In [218]:
input_bert, attention_mask_bert=preprocessing_data(water_problems.clean_text)

In [219]:
input_bert

array([[  101,  2670, 11829, ...,     0,     0,     0],
       [  101,  3815,  5500, ...,     0,     0,     0],
       [  101,  2750,  2714, ...,     0,     0,     0],
       ...,
       [  101,  4661,  2492, ...,     0,     0,     0],
       [  101,  2224,  5300, ...,     0,     0,     0],
       [  101,  5646,  7023, ...,     0,     0,     0]])

In [223]:
embeddings=dbert_model(input_bert,attention_mask_bert )

In [231]:
numpy_embeddings=np.array(embeddings.last_hidden_state[:,0,])

In [232]:
numpy_embeddings.shape

(100, 768)

# Create model

In [268]:
def create_model_img( n_inputs, n_outputs):

    input_layer = keras.Input(shape=(n_inputs, 768))
    
    layer1 = keras.layers.Dense(256, activation='relu')(input_layer)
    dropout = keras.layers.Dropout(0.3)(layer1)
    layer2 = keras.layers.Dense(256, activation='relu')(dropout)

    # output is the next word in sequence
    output_layer = keras.layers.Dense(
        n_outputs, activation='sigmoid')(layer2)
    model = keras.models.Model(
        inputs=input_layer, outputs=output_layer)

    model.summary()

    return model

In [269]:
create_model_img(100,4)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 100, 768)]        0         
                                                                 
 dense_18 (Dense)            (None, 100, 256)          196864    
                                                                 
 dropout_25 (Dropout)        (None, 100, 256)          0         
                                                                 
 dense_19 (Dense)            (None, 100, 256)          65792     
                                                                 
 dense_20 (Dense)            (None, 100, 4)            1028      
                                                                 
Total params: 263,684
Trainable params: 263,684
Non-trainable params: 0
_________________________________________________________________


# Train and test split

In [240]:
water_problems[["pollution","treatment","climate","biomonitoring"]]=water_problems[["pollution","treatment","climate","biomonitoring"]].astype(int)

In [243]:
## get Y data
## get X data
Y=water_problems[["pollution","treatment","climate","biomonitoring"]].to_numpy()
X=numpy_embeddings.copy()

In [244]:
train_X,test_X,train_y,test_y=train_test_split(X,Y,test_size=0.2)

#print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))


In [248]:
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((80, 768), (20, 768), (80, 4), (20, 4))

In [249]:
train_X.shape[-1]

768

# Train model

In [270]:
model=create_model_img(train_X.shape[-1],train_y.shape[-1])
model.compile(optimizer='adam',
                  loss='binary_crossentropy', metrics=["accuracy"])

#callback = keras.callbacks.EarlyStopping(
#    monitor='loss', patience=3)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 768, 768)]        0         
                                                                 
 dense_21 (Dense)            (None, 768, 256)          196864    
                                                                 
 dropout_26 (Dropout)        (None, 768, 256)          0         
                                                                 
 dense_22 (Dense)            (None, 768, 256)          65792     
                                                                 
 dense_23 (Dense)            (None, 768, 4)            1028      
                                                                 
Total params: 263,684
Trainable params: 263,684
Non-trainable params: 0
_________________________________________________________________


In [271]:
model.fit(train_X, train_y, epochs=100, initial_epoch=0,
                                  steps_per_epoch=10)
                                  #callbacks=[callback])

Epoch 1/100
10/10 [==============================] - 1s 7ms/step - loss: 0.4962 - accuracy: 0.5000
Epoch 2/100
10/10 [==============================] - 0s 7ms/step - loss: 0.3889 - accuracy: 0.7625
Epoch 3/100
10/10 [==============================] - 0s 5ms/step - loss: 0.2940 - accuracy: 0.8125
Epoch 4/100
10/10 [==============================] - 0s 5ms/step - loss: 0.2600 - accuracy: 0.7750
Epoch 5/100
10/10 [==============================] - 0s 5ms/step - loss: 0.2362 - accuracy: 0.7625
Epoch 6/100
10/10 [==============================] - 0s 5ms/step - loss: 0.1988 - accuracy: 0.7875
Epoch 7/100
10/10 [==============================] - 0s 8ms/step - loss: 0.1831 - accuracy: 0.7125
Epoch 8/100
10/10 [==============================] - 0s 8ms/step - loss: 0.1641 - accuracy: 0.6625
Epoch 9/100
10/10 [==============================] - 0s 9ms/step - loss: 0.1709 - accuracy: 0.8375
Epoch 10/100
10/10 [==============================] - 0s 9ms/step - loss: 0.1557 - accuracy: 0.6375
Epoch 11/

# Evaluate Model

In [272]:
predictions=model.predict(train_X)
prediction=predictions.round()

In [282]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
def get_evalutaion(y_true,y_pred):
  #converting the multi-label classification to a binary output
    mlb = MultiLabelBinarizer()
    actual = mlb.fit_transform(y_true)
    pred = mlb.fit_transform(y_pred)

    #fitting the data for calculating the f1 score 
    f1 = f1_score(actual, pred, average = "samples")
    accuracy= accuracy_score(y_true, y_pred)
    return f1, accuracy






In [283]:
get_evalutaion(y_true=train_y,y_pred=prediction)

(1.0, 1.0)

In [284]:
accuracy_score(train_y, prediction)

1.0

# Test on test set

In [285]:
predictions_test=model.predict(test_X)
predictions_test=predictions_test.round()

In [286]:
accuracy_score(test_y, predictions_test)

0.55

In [287]:
get_evalutaion(test_y, predictions_test)

(0.9166666666666666, 0.55)